In [2]:

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# import boto3
import pandas as pd
from dotenv import load_dotenv
import os
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
import numpy as np
from mlflow.models.signature import infer_signature
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [15]:
load_dotenv()

access_key = os.getenv("AWS_ACCESS_KEY_ID_CLIENT")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY_CLIENT")
s3_url = os.getenv("MLFLOW_S3_ENDPOINT_URL_CLIENT")
tracker_url = os.getenv("MLFLOW_URL_CLIENT")
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL_CLIENT")
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_ADMIN_USERNAME_CLIENT")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_ADMIN_PASSWORD_CLIENT")
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"
bucket_name = "dataset"
object_name = "water_potability.csv"

In [16]:
print(s3_url)
print(tracker_url)
print(access_key)
print(secret_key)

https://10.34.4.242
http://10.34.4.242
member
F96labjarkom


In [17]:
s3 = boto3.client(
    's3',
    endpoint_url=s3_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    verify=False
)

In [18]:
try:
    # Attempt to list buckets
    response = s3.list_buckets()
    
    # If successful, print bucket names
    print("Connected successfully! Buckets available:")
    for bucket in response['Buckets']:
        print(f"- {bucket['Name']}")
except Exception as e:
    # Print any connection errors
    print("Connection error:", e)


Connected successfully! Buckets available:
- dataset
- mlflow


In [19]:
try:
    response = s3.get_object(Bucket=bucket_name, Key=object_name)
    print(response)
    dataset_content = response.get('Body')
    print(dataset_content)
    # Load the dataset into a DataFrame
    df = pd.read_csv(dataset_content)
    print("Dataset loaded successfully:")
    print(df.head())
except Exception as e:
    print("Error fetching dataset from MinIO:", e)


{'ResponseMetadata': {'RequestId': '1816E336C980E23E', 'HostId': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'accept-ranges': 'bytes', 'content-length': '525187', 'content-type': 'text/csv', 'etag': '"a5eeae83d792e40da6c2f9e1eab945fc"', 'last-modified': 'Thu, 02 Jan 2025 13:27:29 GMT', 'server': 'MinIO', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'Origin, Accept-Encoding', 'x-amz-id-2': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'x-amz-request-id': '1816E336C980E23E', 'x-content-type-options': 'nosniff', 'x-ratelimit-limit': '830', 'x-ratelimit-remaining': '830', 'x-xss-protection': '1; mode=block', 'x-amz-version-id': 'ec0e7df1-0564-498d-b08c-738edbf93673', 'date': 'Thu, 02 Jan 2025 13:28:50 GMT'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2025, 1, 2, 13, 27, 29, tzinfo=tzutc()), 'ContentLength': 525187, 'ETag': '"a5eeae83d792e4

In [20]:
# df = df[['ph','Solids','Turbidity','Potability']]
# df = df[['ph','Solids','Potability']]
df = df[['ph','Potability']]

In [21]:
mlflow.set_tracking_uri(tracker_url)
experiment_name = "water_potability"
mlflow.set_experiment(experiment_name)

data = df.drop('Potability', axis=1)
target = df['Potability']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

2025/01/02 20:28:51 INFO mlflow.tracking.fluent: Experiment with name 'water_potability' does not exist. Creating a new experiment.


In [22]:
from mlflow.models.signature import infer_signature
import mlflow.xgboost
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Initialize and train the model
    model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, model.predict(X_test))

    # Log the model with MLflow
    mlflow.xgboost.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2025/01/02 20:28:52 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2025/01/02 20:28:52 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/Users/fauzanghaza/Applications/miniconda3/envs/MLOps/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [20:28:53] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1733179641447/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


Run ID: 6e7d467fa18145e7bb14a49b3cd349f0
Model accuracy: 0.573170731707317
🏃 View run secretive-pug-675 at: http://10.34.4.242/#/experiments/1/runs/6e7d467fa18145e7bb14a49b3cd349f0
🧪 View experiment at: http://10.34.4.242/#/experiments/1


2025/01/02 20:28:59 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/01/02 20:28:59 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [23]:
from mlflow.models.signature import infer_signature
import mlflow.xgboost
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, model.predict(X_test))

    # Log the model with MLflow
    mlflow.sklearn.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2025/01/02 20:28:59 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2025/01/02 20:28:59 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Run ID: 2336638c424a497895da17eb358dc388
Model accuracy: 0.6234756097560976
🏃 View run suave-ox-814 at: http://10.34.4.242/#/experiments/1/runs/2336638c424a497895da17eb358dc388
🧪 View experiment at: http://10.34.4.242/#/experiments/1


2025/01/02 20:29:06 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/01/02 20:29:06 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [24]:
import mlflow.pyfunc

model_uri = f"runs:/{run_id}/model"
model = mlflow.pyfunc.load_model(model_uri)

# Check if metadata and signature are available
if model.metadata:
    print("Model signature:", model.metadata.get_input_schema())
else:
    print("Model metadata is missing.")

/Users/fauzanghaza/Applications/miniconda3/envs/MLOps/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

Model signature: ['ph': double (optional), 'Solids': double (required)]
